In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re

%matplotlib inline

In [2]:
directory = pd.read_json('directory.json').sort_index()
documents = pd.read_json('documents.json')

In [3]:
directory.head()

,country,lang,more,page,url
0,阿富汗,zh,default_16.shtml,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1,阿富汗,zh,default_46.shtml,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
2,赤道几内亚,zh,default_3.shtml,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
3,阿拉伯石油输出国组织,zh,None,重要文件,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...
4,科特迪瓦,zh,None,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...


In [4]:
documents.head()

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,习近平会见阿富汗总统加尼(2018-06-10),2018-06-10
10,尼泊尔,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_尼泊尔_驻外报道_,第八届中国高等教育展在加德满都揭幕(2013-05-14),2013-05-14
100,尼泊尔,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_尼泊尔_驻外报道_,驻尼泊尔大使杨厚兰接受新华社采访(2013-01-15),2013-01-15
1000,哥斯达黎加,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_北美洲_哥斯达黎加_驻外报道_,驻哥斯达黎加大使汤恒拜会哥新任第一副总统兼外长坎贝尔(2018-05-17),2018-05-17


In [5]:
directory.shape

(1532, 5)

In [6]:
documents.shape

(125745, 8)

In [7]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options, executable_path="geckodriver.exe")
print("Firefox Headless Browser Invoked")

Firefox Headless Browser Invoked


In [8]:
try:
    subdirectory_progress = pd.read_json('subdirectory_progress.json')
except:
    subdirectory_progress = pd.DataFrame(columns=['sub_url'])
    
try:
    directory_progress = pd.read_json('directory_progress.json')
except:
    directory_progress = pd.DataFrame(columns=['dir_url'])

In [9]:
subdirectory_progress.head()

,sub_url
0,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
10,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
100,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1000,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...


In [10]:
subdirectory_progress.shape

(8220, 1)

In [11]:
directory_progress.shape

(452, 1)

In [12]:
directory.loc[directory['more'].notnull(),].shape

(471, 5)

In [13]:
counter = 0
dir_counter = 0
problems = []
for i in directory.loc[directory['more'].notnull(),'more'].index:
    try:
        url_root = directory.loc[i,'url']
        if url_root in list(directory_progress['dir_url']):
            print('SKIP DIR')
            continue
        else:
            print('\n','ROOT')
            print(url_root)
            print('\n','SUBS')
            country = directory.loc[i,'country']
            page = directory.loc[i,'page']
            lang = directory.loc[i,'lang']

            subpage_count = int(re.split(r'_|\.',directory.loc[i,'more'])[1])
            subpages = np.arange(subpage_count)+1
            subpages_urls = [(url_root+'default_'+str(m)+'.shtml') for m in subpages]
            
            dir_counter += 1

            for h in subpages_urls:
                if h in list(subdirectory_progress['sub_url']):
                    print('SKIP SUB')
                    continue
                else:                
                    print(h,'\n')
                    driver.get(h)
                    soup=BeautifulSoup(driver.page_source, 'lxml')

                    nav = str()
                    for k in soup.find(class_='nav').find_all('a'):
                        nav = nav + k.text + '_'

            #         try:
            #             more = soup.find("a", string='尾页')['href']
            #             print(more)
            #             directory.loc[i,'more'] = more
            #         except:
            #             print('NO MORE')

                    for j in soup.find(class_='rebox_news').find_all('li'):
                        title = j.text
                        url_new = url_root+j.find('a')['href'][2:]
                        print(url_new)

                        try:
                            date = re.findall(r"\d\d\d\d\-\d\d\-\d\d",j.text)[0]
                        except:
                            date = '9999-99-99'
                        print(date)

                        documents = documents.append({'country':country,
                                                      'page':page,
                                                      'url':url_new,
                                                      'lang':lang,
                                                      'nav':nav,
                                                      'title':title,
                                                      'date':date,
                                                     },
                                                     ignore_index=True)
                        subdirectory_progress = subdirectory_progress.append({'sub_url':h},ignore_index=True)
                        counter += 1
                        print(dir_counter)
                        print(counter,'\n')
                        
                directory_progress = directory_progress.append({'dir_url':url_root},ignore_index=True)
    except:
        problems.append(i)


 ROOT
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/xgxw_676213/

 SUBS
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB

 ROOT
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/1206x2_676229/

 SUBS
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB
SKIP SUB

 ROOT
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677510/xgxw_677516/

 SUBS
SKIP SUB
SKIP SUB
SKIP SUB
SKIP DIR

 ROOT
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/dyfheas_682566/xgxw_682572/

 SUBS
SK

In [14]:
documents.shape

(125745, 8)

In [15]:
documents.tail()

,country,lang,more,page,url,nav,title,date
99995,刚果(布),zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_刚果(布)_驻外报道_,驻刚果（布）大使关键向刚外长递交国书副本(2012-05-22),2012-05-22
99996,刚果(布),zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_刚果(布)_驻外报道_,驻刚果（布）使馆临时代办段金柱出席刚国家二号公路一期工程竣工暨二期工程开工典礼(2012-0...,2012-05-21
99997,刚果(布),zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_刚果(布)_驻外报道_,驻刚果（布）大使李树立辞行拜会萨苏总统(2012-04-26),2012-04-26
99998,刚果(布),zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_刚果(布)_驻外报道_,驻刚果（布）大使李树立举行离任招待会(2012-04-26),2012-04-26
99999,刚果(布),zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_刚果(布)_驻外报道_,刚果（布）外长为李树立大使送行(2012-04-22),2012-04-22


In [16]:
documents.shape

(125745, 8)

In [17]:
documents = documents.drop_duplicates()
subdirectory_progress = subdirectory_progress.drop_duplicates()
directory_progress = directory_progress.drop_duplicates()

In [18]:
documents.shape

(125745, 8)

In [19]:
documents.to_json('documents.json')

In [20]:
subdirectory_progress.to_json('subdirectory_progress.json')

In [21]:
directory_progress.to_json('directory_progress.json')

In [22]:
pd.Series(problems).to_json('problems.json')

In [23]:
problems

[]

In [24]:
type(documents.loc[1,'url'])

str